In [483]:
import pandas as pd
data = pd.read_csv("/content/sample_data/EMPLOYEE.csv")
jobdesc = pd.read_csv("/content/sample_data/JOB - JOB.csv")
course = pd.read_csv("/content/sample_data/courses - courses.csv")

In [484]:
data.head

<bound method NDFrame.head of                 Name  Experience           Job Role  Gender  \
0       Alice Turner           7  Software Engineer  Female   
1          Bob Evans           3       Data Analyst    Male   
2        Emma Foster           5     UI/UX Designer  Female   
3        Jack Murphy           2   Junior Developer    Male   
4       Sophia Adams           9     Data Scientist  Female   
..               ...         ...                ...     ...   
253  Benjamin Turner           9    DevOps Engineer    Male   
254     Harper Adams           2    Content Creator  Female   
255   Henry Martinez           7       Data Analyst    Male   
256      Grace Baker           4        UX Designer  Female   
257    Samuel Turner           1   Junior Developer    Male   

                                    Skills    Phone Number  \
0                     Java Python HTML CSS  (111) 123-4567   
1                  Data Analysis SQL Excel  (222) 234-5678   
2        UI/UX Design Wiref

In [485]:
data.shape

(258, 8)

In [486]:
data = data.drop_duplicates()

In [487]:
data.shape


(258, 8)

#Cosine Similarity after vectorization of Texts

In [488]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def match_predict(t1,t2):
    cv = CountVectorizer()
    text=[t1,t2]
    countmatrix = cv.fit_transform(text)
    matchpercent= cosine_similarity(countmatrix)[0][1] * 100
    matchpercent= round(matchpercent,2)
    return(matchpercent)

#NLP ALGORITHM
 A model to identify important keywords to extract required skills from a job description using CountVectorizer,Text Frequency(TF) and Inverse Document Frequency(IDF)

In [489]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
from nltk.corpus import stopwords
import re

def converttokeywords(text):


  # Preprocess and tokenize text
  def preprocess(text):
      text = text.lower()
      text = re.sub(r'\d+', '', text)  # Remove numbers
      text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
      return text

  preprocessed_text = preprocess(text)

  # Tokenize and remove stopwords
  stop_words = set(stopwords.words("english"))
  tokenized_text = nltk.word_tokenize(preprocessed_text)
  filtered_text = [word for word in tokenized_text if word not in stop_words]

  # Convert tokens back to text
  filtered_text_str = " ".join(filtered_text)

  # Create a CountVectorizer
  vectorizer = CountVectorizer()

  # Fit and transform the text data
  X = vectorizer.fit_transform([filtered_text_str])

  # Create a TfidfTransformer
  tfidf_transformer = TfidfTransformer()

  # Transform the count matrix to TF-IDF
  tfidf_matrix = tfidf_transformer.fit_transform(X)

  # Get feature names (words)
  feature_names = vectorizer.get_feature_names_out()

  # Get TF-IDF scores for each feature (word)
  tfidf_scores = tfidf_matrix.toarray()[0]

  # Sort and select top keywords
  num_keywords = 7;
  sorted_indices = tfidf_scores.argsort()[-num_keywords:][::-1]
  top_keywords = [feature_names[idx] for idx in sorted_indices]

  return top_keywords


#Testing Extraction of Keywords from Job Expectations


In [490]:
for i in jobdesc['Expectations']:
  print(converttokeywords(i))

['knowledge', 'python', 'data', 'visulalization', 'sql', 'libraries', 'expert']
['java', 'swing', 'knowledge', 'jdbc', 'expert']
['marketing', 'knowledge', 'great', 'finance', 'expert', 'digital']
['knowledge', 'great', 'graphics', 'front', 'frameworks', 'end', 'designing']
['strong', 'preferred', 'networking', 'knowledge', 'ccna']
['knowledge', 'sql', 'python', 'great', 'excel', 'data', 'analysis']
['java', 'swing', 'knowledge', 'jdbc', 'expert']
['seo', 'knowledge', 'great', 'copywriting', 'blogging']
['strong', 'skills', 'people', 'management', 'great', 'expertise', 'communication']
['knowledge', 'great', 'works', 'ux', 'ui', 'react', 'javascript']
['sql', 'nosql', 'knowledge', 'expert', 'databases', 'containerization']
['skills', 'handling', 'great', 'customer', 'communication', 'capablities']
['skills', 'knowledge', 'finance', 'communication']
['sql', 'shoud', 'knowledge', 'frameworks', 'familier', 'excel', 'data']
['expert', 'strategy', 'skills', 'relations', 'recruitment', 'prod

#Using Regular Expressions to search for unmatched keyword in Job Description

In [491]:
import re

def neededskills(text,keywords):

  # Perform string matching for each keyword
  unmatched_keywords = [keyword for keyword in keywords if not re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE)]

  return(unmatched_keywords)

In [492]:
applicantskill =data.iloc[0,data.columns.get_loc('Skills')].lower()
jobskill = ' '.join(map(str,converttokeywords(jobdesc.iloc[0,jobdesc.columns.get_loc('Expectations')])))
print(match_predict(applicantskill,jobskill))

18.9


In [493]:
applicantskill =data.iloc[0,data.columns.get_loc('Skills')].lower()
jobskill = converttokeywords(jobdesc.iloc[0,jobdesc.columns.get_loc('Expectations')])
print(neededskills(applicantskill,jobskill))

['knowledge', 'data', 'visulalization', 'sql', 'libraries', 'expert']


#Finding Company and Job roles for the applicant skillset

In [494]:
def findjobs(applicantskill):
  applicantskill = applicantskill.lower()
  companyroles= []
  for i in range(jobdesc.shape[0]):
    jobskill = ' '.join(map(str,converttokeywords(jobdesc.iloc[i,jobdesc.columns.get_loc('Expectations')])))
    if(match_predict(applicantskill,jobskill)>20):

      companyroles.append([jobdesc.iloc[i,jobdesc.columns.get_loc('Company')],jobdesc.iloc[i,jobdesc.columns.get_loc('JobTitle')]])
  return companyroles
  # return companyroles

In [495]:
applicantskill =data.iloc[0,data.columns.get_loc('Skills')]
print(findjobs(applicantskill))

[['Google', 'Software Engineer'], ['Citi', 'Front-end Developer']]


#Ranking of candidates based on Cosine similarity between JOB expectations

In [496]:
def rankcandidates(jobskill):
  ranklist = []
  jobskill = ' '.join(map(str,converttokeywords(jobskill)))
  for i in range(data.shape[0]):
    ranklist.append([data.iloc[i,data.columns.get_loc("Name")],match_predict(data.iloc[i,data.columns.get_loc('Skills')].lower(),jobskill)])
  ranklist.sort(key = lambda x : x[1],reverse = True)
  return ranklist


In [497]:
jobskills = jobdesc.iloc[9,jobdesc.columns.get_loc('Expectations')]
print(jobskills)
print(rankcandidates(jobskills))

Great knowledge in UI or UX. Should be expert in front end frame works. Great Knowledge in HTML, CSS, JavaScript, React or Angular.
[['Jack Murphy', 37.8], ['Liam Murphy', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Liam Turner', 37.8], ['Emma Foster', 30.86], ['Chloe Turner', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Chloe Martinez', 30.86], ['Alice Turner', 0.0], ['Bob Evans', 0.0], ['Sophia Adams', 0.0], ['Liam Turner', 0.0], ['Olivia Cook', 0.0], ['Ethan Martinez', 0.0], ['Mia Turner', 0.0], ['Lucas Foster', 0.0], ['Amelia Adams', 0.0], ['Benjamin Turner', 0.0], ['Harper Turner', 0.0], ['Henry Cook', 0.0],

#Adding New Skills to Applicant Database and Finding Jobs


In [498]:
applicantskill =data.iloc[1,data.columns.get_loc('Skills')]
print(findjobs(applicantskill))

[['IBM', 'Data Scientist'], ['TCS', 'Data Analyst'], ['Cisco', 'DevOps Engineer'], ['Infosys', 'Backend Developer'], ['Meta', 'Database Administrator']]


In [499]:
def addskill(applemail,skill):
  data.loc[data['Email']==applemail, "Skills"] = data.loc[data['Email']==applemail, "Skills"]+" "+skill

  data.to_csv("EMPLOYEE.csv", index=False)

In [500]:
addskill("bob.evans@email.com","SQL")
data

,Name,Experience,Job Role,Gender,Skills,Phone Number,Email,Employment Status
0,Alice Turner,7,Software Engineer,Female,Java Python HTML CSS,(111) 123-4567,alice.turner@email.com,Employed
1,Bob Evans,3,Data Analyst,Male,Data Analysis SQL Excel SQL,(222) 234-5678,bob.evans@email.com,Employed
2,Emma Foster,5,UI/UX Designer,Female,UI/UX Design Wireframing Adobe XD,(333) 345-6789,emma.foster@email.com,Employed
3,Jack Murphy,2,Junior Developer,Male,HTML CSS JavaScript React,(444) 456-7890,jack.murphy@email.com,Unemployed
4,Sophia Adams,9,Data Scientist,Female,Python Machine Learning Data Analysis,(555) 567-8901,sophia.adams@email.com,Employed
...,...,...,...,...,...,...,...,...
253,Benjamin Turner,9,DevOps Engineer,Male,CI/CD Docker AWS Jenkins,(222) 456-7890,benjamin.turner@email.com,Employed
254,Harper Adams,2,Content Creator,Female,Video Editing Social Media SEO,(333) 567-8901,harper.adams@email.com,Employed
255,Henry Martinez,7,Data Analyst,Male,Data Cleaning Visualization Excel,(444) 678-9012,henry.martinez@email.com,Employed
256,Grace Baker,4,UX Designer,Female,Wireframing User Research Prototyping,(555) 789-0123,grace.baker@email.com,Employed


In [501]:
print(data.iloc[1,data.columns.get_loc('Skills')])
print(findjobs(data.iloc[1,data.columns.get_loc('Skills')]))

Data Analysis SQL Excel SQL
[['IBM', 'Data Scientist'], ['TCS', 'Data Analyst'], ['Cisco', 'DevOps Engineer'], ['Infosys', 'Backend Developer'], ['Meta', 'Database Administrator']]


#Suggestion of Courses with respect to Required Skills

In [502]:
applicantskill =data.iloc[7,data.columns.get_loc('Skills')].lower()
jobskill = converttokeywords(jobdesc.iloc[0,jobdesc.columns.get_loc('Expectations')])
neededskill = neededskills(applicantskill,jobskill)

In [503]:
def suggest_courses(neededskill):
  neededskill = ' '.join(map(str,neededskill))
  mentorcourses = []
  for i in range(course.shape[0]):
    if(match_predict(neededskill,course.iloc[i,course.columns.get_loc("Skills_Developed")])>30):
      mentorcourses.append([course.iloc[i,course.columns.get_loc("Course")],course.iloc[i,course.columns.get_loc("Instructor")]])
  return mentorcourses

In [504]:
print(suggest_courses(neededskill))

[['Data Analysis with Pandas and NumPy', 'Jane Doe'], ['Machine Learning for Beginners', 'Dr. Emily Adams']]
